**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [39]:
!sudo pip install scikit-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, BatchNormalization, Dropout, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function *act* chooses an action to take given the current state.

If we don't use ```epsilon```, then the model would choose the action that gives the best reward given what it learnt and never try another action.
The idea is that in the proportion ```epsilon``` of the trials the model will choose an action randomly and gain information, and in the proportion ```1 - epsilon``` of the trials it would stick to the learnt policy and take the best action according to it.
In the end the greater ```epsilon``` the more explorative the model will be, but the less greedy it will be.
We call this the exploration exploitation dilemma.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50 # set small when debugging
epochs_test = 50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` gives the position of the mouse on the island. In particular it is set to 1 where the mouse is and 0 everywhere else, except on the extra cells at the edge of the island where it is set to -1. This helps the model to understand where it is and where it can go.

The array ```board``` is generated randomly at the beginning of the game by the method *reset*. It contains the reward (bonus or malus) of each cell. That is when the mouse goes on a cell, it will receive a reward given by the value in the ```board``` array at this position. Then when the mouse leaves the cell it's value is turned to 0 (as the poisonous or edible cheese was eaten). This helps the model to understand what gives a bonus and what gives a malus.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 4, 1)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state) # only act according to the learnt policy at test time

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [47]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/20.0. Average score (-8.0)
Win/lose count 17.0/18.0. Average score (-4.5)
Win/lose count 13.5/11.0. Average score (-2.1666666666666665)
Win/lose count 9.5/18.0. Average score (-3.75)
Win/lose count 10.0/12.0. Average score (-3.4)
Win/lose count 13.0/16.0. Average score (-3.3333333333333335)
Win/lose count 17.5/16.0. Average score (-2.642857142857143)
Win/lose count 11.5/13.0. Average score (-2.5)
Win/lose count 9.0/17.0. Average score (-3.111111111111111)
Win/lose count 14.0/10.0. Average score (-2.4)
Win/lose count 10.5/10.0. Average score (-2.1363636363636362)
Win/lose count 7.5/13.0. Average score (-2.4166666666666665)
Win/lose count 9.5/14.0. Average score (-2.576923076923077)
Win/lose count 11.0/15.0. Average score (-2.6785714285714284)
Win/lose count 10.0/25.0. Average score (-3.5)
Win/lose count 14.5/8.0. Average score (-2.875)
Win/lose count 6.5/3.0. Average score (-2.5)
Win/lose count 10.5/17.0. Average score (-2.7222222222222223)
Win/lose count 6.5/4.0. Av

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




### 1)
Using the Markov property and a change of variable on the time index we get:
\begin{align}
Q^{\pi}(s,a) &= E_{p^{\pi}}[\sum_{t\geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&= r(s,a) + \gamma \sum_{(s', a')} p(s', a' | s, a) E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_0=s, a_0=a, s_1=s', a_1=a'] \\
&= r(s,a) + \gamma \sum_{(s', a')} p(s', a' | s, a) E_{p^{\pi}}[\sum_{t\geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_1=s', a_1=a'] \\
&= r(s,a) + \gamma \sum_{(s', a')} p(s', a' | s, a) E_{p^{\pi}}[\sum_{t'\geq 0}\gamma^{t'}r(s_{t'},a_{t'})|s_{0}=s', a_{0}=a'] \\
&= r(s,a) + \gamma  E_{(s', a') \sim p(., .| s, a)}[Q^{\pi}(s', a')] \\
&= E_{(s',a')\sim p(., .|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

### 2)
We introduce call $\pi^* = arg\max_\pi Q^{\pi}(s,a)$. In particular we have: $\pi^*(a|s) = \mathbb{I}\{a =  arg\max_{a'} Q^*(s,a')\}$. That is the optimal policy always chooses the action that maximizes $Q$ for a given $s$. Then we have:
\begin{align}
Q^{*}(s,a) &= \max_\pi Q^{\pi}(s,a) \\
&= \max_\pi E_{(s',a')\sim p(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= E_{(s',a')\sim p^*(.,.|s,a)}[r(s,a) + \gamma Q^{*}(s',a')] \\
&= E_{s'\sim p^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align}

### 3)
A plausible objective would try to minimize on average under the best policy the error of the learnt $Q_\theta$ function. That is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p^*(.|s,a)}\Vert r+\gamma \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
from random import choices

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            del self.memory[int(np.random.randint(0, self.max_memory, 1))] # randomly delete an element
            #self.memory = self.memory[1:] # first in first out
            self.memory.append(m)
        else:
            self.memory.append(m)

    def random_access(self, k):
        return choices(self.memory, k=k)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose
        
        if e % 5 == 0:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e, epoch, loss, win, lose, win-lose))
        
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        memory = self.memory.random_access(self.batch_size)
        input_next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        for i in range(self.batch_size):
            input_states[i] = memory[i][0]
            input_next_states[i] = memory[i][1]
        
        # it is faster to predict on a sequence rather to do a sequence of predictions
        target_q = self.model.predict(input_states) # then the gradients will be 0 were we don't know the reward
        next_q = self.model.predict(input_next_states) # Q value of the next state
        
        for i in range(self.batch_size): # update with the true value of the reward where available
            if memory[i][4]:
                target_q[i, memory[i][2]] = memory[i][3] # if the game is over Q = r
            else:
                target_q[i, memory[i][2]] = memory[i][3] + self.discount*next_q[i].max()
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)

        ####### FILL IN
        # NN Model made of 3 fully connected layers
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(.2))
        model.add(Dense(16))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(.2))
        model.add(Dense(self.n_action))
        model.compile(adam(lr=lr), "mse")
        
        self.model = model

In [51]:
epochs_train = 250
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0005, epsilon = 0.1, memory_size=5000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))

Epoch 000/250 | Loss 0.4188 | Win/lose count 8.0/6.0 (2.0)
Epoch 005/250 | Loss 0.0609 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/250 | Loss 0.0166 | Win/lose count 6.5/11.0 (-4.5)
Epoch 015/250 | Loss 0.0150 | Win/lose count 2.5/2.0 (0.5)
Epoch 020/250 | Loss 0.0065 | Win/lose count 4.5/2.0 (2.5)
Epoch 025/250 | Loss 0.0081 | Win/lose count 9.5/6.0 (3.5)
Epoch 030/250 | Loss 0.0015 | Win/lose count 3.0/4.0 (-1.0)
Epoch 035/250 | Loss 0.0104 | Win/lose count 4.5/0 (4.5)
Epoch 040/250 | Loss 0.0112 | Win/lose count 7.5/5.0 (2.5)
Epoch 045/250 | Loss 0.0092 | Win/lose count 5.0/6.0 (-1.0)
Epoch 050/250 | Loss 0.0140 | Win/lose count 4.5/4.0 (0.5)
Epoch 055/250 | Loss 0.0092 | Win/lose count 7.5/3.0 (4.5)
Epoch 060/250 | Loss 0.0049 | Win/lose count 6.5/3.0 (3.5)
Epoch 065/250 | Loss 0.0167 | Win/lose count 5.5/4.0 (1.5)
Epoch 070/250 | Loss 0.0091 | Win/lose count 11.5/5.0 (6.5)
Epoch 075/250 | Loss 0.0139 | Win/lose count 10.5/5.0 (5.5)
Epoch 080/250 | Loss 0.0147 | Win/lose count 9.5/5.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)

        ###### FILL IN
        # Two fully convolutional layers and 2 fully connected layers

        model = Sequential()
        
        model.add(Conv2D(16, 3, padding="same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(AveragePooling2D((2, 2), padding="same"))
        
        model.add(Conv2D(32, 3, padding="same"))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D((2, 2), padding="same"))
        
        model.add(Flatten())
        model.add(Dropout(.2))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(.2))
        model.add(Dense(self.n_action))
        
        model.compile(adam(lr=lr), "mse")
        
        self.model = model

In [53]:
epochs_train = 250
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.0005, epsilon = 0.1, memory_size=5000, batch_size = 64)
train(agent, env,epochs_train, prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))

Epoch 000/250 | Loss 0.2094 | Win/lose count 4.0/7.0 (-3.0)
Epoch 005/250 | Loss 0.0308 | Win/lose count 4.5/4.0 (0.5)
Epoch 010/250 | Loss 0.0095 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/250 | Loss 0.0075 | Win/lose count 3.0/2.0 (1.0)
Epoch 020/250 | Loss 0.0025 | Win/lose count 3.0/1.0 (2.0)
Epoch 025/250 | Loss 0.0082 | Win/lose count 8.0/4.0 (4.0)
Epoch 030/250 | Loss 0.0040 | Win/lose count 3.0/3.0 (0.0)
Epoch 035/250 | Loss 0.0009 | Win/lose count 12.0/6.0 (6.0)
Epoch 040/250 | Loss 0.0060 | Win/lose count 8.5/1.0 (7.5)
Epoch 045/250 | Loss 0.1007 | Win/lose count 7.5/5.0 (2.5)
Epoch 050/250 | Loss 0.0071 | Win/lose count 6.5/8.0 (-1.5)
Epoch 055/250 | Loss 0.0068 | Win/lose count 3.5/3.0 (0.5)
Epoch 060/250 | Loss 0.0128 | Win/lose count 5.5/1.0 (4.5)
Epoch 065/250 | Loss 0.0050 | Win/lose count 7.0/1.0 (6.0)
Epoch 070/250 | Loss 0.0075 | Win/lose count 3.5/1.0 (2.5)
Epoch 075/250 | Loss 0.0142 | Win/lose count 4.0/2.0 (2.0)
Epoch 080/250 | Loss 0.0051 | Win/lose count 10.0/3.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [54]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 64)
agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 7.0/0. Average score (7.0)
Win/lose count 4.5/0. Average score (5.75)
Win/lose count 1.0/0. Average score (4.166666666666667)
Win/lose count 5.5/0. Average score (4.5)
Win/lose count 15.0/1.0. Average score (6.4)
Win/lose count 14.5/0. Average score (7.75)
Win/lose count 8.5/0. Average score (7.857142857142857)
Win/lose count 0.5/0. Average score (6.9375)
Win/lose count 4.0/0. Average score (6.611111111111111)
Win/lose count 7.0/2.0. Average score (6.45)
Win/lose count 5.5/1.0. Average score (6.2727272727272725)
Win/lose count 0/0. Average score (5.75)
Win/lose count 1.0/0. Average score (5.384615384615385)
Win/lose count 4.5/0. Average score (5.321428571428571)
Win/lose count 0.5/0. Average score (5.0)
Win/lose count 7.0/0. Average score (5.125)
Win/lose count 2.5/0. Average score (4.970588235294118)
Win/lose count 12.5/1.0. Average score (5.333333333333333)
Win/lose count 5.5/0. Average score (5.342105263157895)
Win/lose count 8.5/1.0. Average score (5.

In [0]:
#HTML(display_videos('cnn_test10.mp4'))

In [0]:
#HTML(display_videos('fc_test10.mp4'))

### Analysis

One behaviour I observed is that the mouse would stop exploring when there is no cheese around it, and keep moving backward and forward. Then as the memory buffer has a limited size and as I initially used *first in first out* method of queuing, it would mostly contain examples whith empty cells and the model learns nothing new. That's why I decided to increase the buffer's size and to randomly delete its elements when it is full in order to keep more different examples in memory.

Both algorithms would oscillate a lot during training. As the exploration is not optimal (only 10\% of the moves try to explore) the model doesn't learn efficiently. With a higher temperature it can learn more as it has more information, but with a low temperature it would easily get stuck.

In the end the convolutional neural networks is better as it is more suited for this type of data: it can detect a patern in the neighborhood of the mouse rather than a exact structure of the neighborhood (as is it the case with a fully connected layer).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    # Decreasing epsilon
    # Linear decay for 3/4 of the epochs
    # last 1/4 epochs with epsilon = .1
    epsilons = np.concatenate((np.linspace(1, .1, 3*(epoch//4)), .1*np.ones(epoch - 3*(epoch//4))))

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        #set epsilon
        agent.set_epsilon(epsilons[e])

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        if e % 5 == 0:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{:.1f} ({:.1f})"
                  .format(e, epoch, loss, win, lose, win-lose))
        
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, malus_pos=.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)

        self.malus_pos = malus_pos # malus of already visited cells
        self.malus_position = np.zeros((grid_size,grid_size)) # grid of already visited cells
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # reset malus and set its value where the mouse start
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = self.malus_pos

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
            self.malus_position.reshape(self.grid_size, self.grid_size, 1),
            self.board.reshape(self.grid_size, self.grid_size, 1),
            self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train: # penalize only during training to have a coherent score at test time
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = self.malus_pos # upfate malus on visited cell
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((
            self.malus_position.reshape(self.grid_size, self.grid_size, 1),
            self.board.reshape(self.grid_size, self.grid_size, 1),
            self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

In [59]:
# Training
epochs_train = 300
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, malus_pos=.2)
agent = DQN_CNN(size, lr=.0004, epsilon=0.1, memory_size=5000, batch_size=64, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/300 | Loss 0.6767 | Win/lose count 8.5/42.2 (-33.7)
Epoch 005/300 | Loss 0.0791 | Win/lose count 9.5/41.6 (-32.1)
Epoch 010/300 | Loss 0.0317 | Win/lose count 6.5/43.6 (-37.1)
Epoch 015/300 | Loss 0.0355 | Win/lose count 13.0/41.4 (-28.4)
Epoch 020/300 | Loss 0.0219 | Win/lose count 13.0/36.0 (-23.0)
Epoch 025/300 | Loss 0.0311 | Win/lose count 11.0/45.8 (-34.8)
Epoch 030/300 | Loss 0.0238 | Win/lose count 9.5/36.4 (-26.9)
Epoch 035/300 | Loss 0.0374 | Win/lose count 11.5/36.0 (-24.5)
Epoch 040/300 | Loss 0.0215 | Win/lose count 15.0/36.2 (-21.2)
Epoch 045/300 | Loss 0.0188 | Win/lose count 16.0/33.0 (-17.0)
Epoch 050/300 | Loss 0.0258 | Win/lose count 14.0/40.0 (-26.0)
Epoch 055/300 | Loss 0.0259 | Win/lose count 15.5/35.6 (-20.1)
Epoch 060/300 | Loss 0.0648 | Win/lose count 15.0/37.6 (-22.6)
Epoch 065/300 | Loss 0.0375 | Win/lose count 14.0/38.6 (-24.6)
Epoch 070/300 | Loss 0.0355 | Win/lose count 15.0/41.2 (-26.2)
Epoch 075/300 | Loss 0.0352 | Win/lose count 11.0/37.8 (-26

In [60]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.5/4.0. Average score (10.5)
Win/lose count 9.5/4.0. Average score (8.0)
Win/lose count 17.0/4.0. Average score (9.666666666666666)
Win/lose count 16.5/5.0. Average score (10.125)
Win/lose count 14.5/1.0. Average score (10.8)
Win/lose count 10.0/2.0. Average score (10.333333333333334)
Win/lose count 0.5/0. Average score (8.928571428571429)
Win/lose count 12.0/5.0. Average score (8.6875)
Win/lose count 13.5/4.0. Average score (8.777777777777779)
Win/lose count 16.5/1.0. Average score (9.45)
Win/lose count 18.5/7.0. Average score (9.636363636363637)
Win/lose count 19.5/4.0. Average score (10.125)
Win/lose count 10.5/3.0. Average score (9.923076923076923)
Win/lose count 13.5/2.0. Average score (10.035714285714286)
Win/lose count 14.0/2.0. Average score (10.166666666666666)
Win/lose count 7.5/0. Average score (10.0)
Win/lose count 11.0/0. Average score (10.058823529411764)
Win/lose count 10.0/3.0. Average score (9.88888888888889)
Win/lose count 10.0/0. Average score (9.894

### Analysis

Having a high $\epsilon$ in the beginning allows the model to learn from various situations that it wouldn't have tried by itself.

Then adding a malus on already visited cells forces it to explore the board and eat more cheese.

We can see that these 2 updates lead to a better cover of the grid and a better score.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***